## 1. 필요 Library 설치

In [1]:
!pip install beautifulsoup4

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
!pip install python-telegram-bot

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## 2. 실행 코드

In [1]:
from bs4 import BeautifulSoup
import telegram

import requests

import datetime 
import time

import re

import pandas as pd
from pandas.plotting import table 

import matplotlib.pyplot as plt
from matplotlib import font_manager

In [3]:
my_token = '983160285:AAH5_-HPQuGYRycHMfZ3vmGg7lJEWjqeTE0' 

In [4]:
bot = telegram.Bot(token = my_token) 
chat_id = bot.getUpdates()[-1].message.chat.id

In [5]:
def hot_rank_stock_name():
    
    url = "https://finance.naver.com/sise/lastsearch2.nhn"
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    rank = soup.select('.no')
    stockname = soup.select('.tltle') # i 가 아니라 소문자 L
    etc_total = soup.select('.number')
    
    s_rank = []
    s_name = []
    s_ratio = []
    s_price = []
    s_updown_ratio = []
    s_count = []

    r = re.compile('\d+.\d+%')
    
    for i in range(10) :
        
        s_rank.append(rank[i].text)
        s_name.append(stockname[i].text)
        s_ratio.append(etc_total[i*10].text)
        s_price.append(etc_total[i*10+1].text)
        
        match = r.search(etc_total[i*10+3].text)
        s_updown_ratio.append(match.group())
        
        s_count.append(etc_total[i*10+4].text)
        
    df = pd.DataFrame({"검색순위":s_rank,"종목명":s_name,"검색비율":s_ratio,"현재가":s_price,"등락률":s_updown_ratio,"거래량":s_count})
    
    return df

In [6]:
font_list = font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
print('사용가능한 TTF 폰트 개수:', len(font_list))
font_list[:10] 

사용가능한 TTF 폰트 개수: 745


['C:\\Windows\\Fonts\\GARAIT.TTF',
 'C:\\Windows\\Fonts\\BELLB.TTF',
 'C:\\Windows\\Fonts\\couri.ttf',
 'C:\\Windows\\Fonts\\HARLOWSI.TTF',
 'C:\\Windows\\Fonts\\MT.TTF',
 'C:\\Windows\\Fonts\\seguisbi.ttf',
 'C:\\Windows\\Fonts\\framd.ttf',
 'C:\\Windows\\Fonts\\IMPRISHA.TTF',
 'C:\\Windows\\Fonts\\constani.ttf',
 'C:\\Windows\\Fonts\\LBRITEDI.TTF']

In [7]:
font_fname = 'C:\\Windows\\Fonts\\HANDotumB.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()
plt.rcParams["font.family"] = font_family

In [ ]:
if __name__ == "__main__" :
        
    while True : 
        
        now_hour = datetime.datetime.now().hour
        today = datetime.datetime.now().strftime('%Y.%m.%d')
        yesterday = (datetime.datetime.now() + datetime.timedelta(-1)).strftime('%Y.%m.%d')
        
        if 17 <= now_hour < 18 or 12 <= now_hour < 13 :
            
            df = hot_rank_stock_name() 
            df.index = df['검색순위']
            df.drop(columns='검색순위', inplace=True)
            
            fig, ax = plt.subplots(figsize=(15, 5)) 
            ax.xaxis.set_visible(False) 
            ax.yaxis.set_visible(False) 
            ax.set_frame_on(False)
            tabla = table(ax, df, loc='center', colWidths=[0.13]*len(df.columns)) 
            tabla.set_fontsize(20) 
            tabla.scale(1.4, 2) 
            plt.savefig('table.png', transparent=True)
            
            bot.sendMessage(chat_id = chat_id, text= 'Naver 종목 검색순위' + '-' + datetime.datetime.now().strftime('%Y.%m.%d %Hh %Mm'))
            bot.send_photo(chat_id=chat_id, photo=open('table.png', 'rb'))
          
        time.sleep(60)        